<a href="https://colab.research.google.com/github/Penguin2611/Aspect-based-summarization-of-reviews/blob/master/Text_Preprocessing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
from google.colab import drive
drive.mount('/content/drive',force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
import nltk
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string
import gensim 
from gensim.models import Word2Vec

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
## Tokenized Output
## object creation
# stop_words = set(stopwords.words('english'))
# file = open('/content/drive/My Drive/amazon_review.txt')

def preprocess(s):
### Data Reading
  # z = file.readlines()
  # s = ""
  # s = s.join(z)


  ### Splitting input lines
  ### Tokenizing line by line
  stop_words = set(stopwords.words('english'))
  l = WordNetLemmatizer()
  t = TweetTokenizer() 
  for c in string.punctuation:   ### Removing Punctuations
      if c == string.punctuation[13]:
        s = s.replace(c," ")
      if c != string.punctuation[6] and c != string.punctuation[12]:
        s= s.replace(c,"")

  res = s.splitlines()
  print(res) 
  text = [] 
  for i in res:
    p = t.tokenize(i) ### Tokenization
    text.append(p)
    text.append('\n')

  ## Flatting the list

  flat_list = []
  for sublist in text:
      for item in sublist:
          flat_list.append(item)

  tokens = [token.lower() for token in flat_list]
 


  ### Combining list elements to string
  ### Tokenized output

  ### Lemmization
  lem_text =[]
  for c in tokens:
    tokenize_text = l.lemmatize(c,pos='v')
    tokenize_text = l.lemmatize(tokenize_text,pos='n')
    tokenize_text = l.lemmatize(tokenize_text,pos='a')
    lem_text.append(tokenize_text)

  ### Stopwords Removal
  stop_text = []
  for w in lem_text: 
      if w not in stop_words: 
          stop_text.append(w)
  # print("Processed Tokens",stop_text)
  tagged = nltk.pos_tag(stop_text) ### POS Tagging 
  return tagged

  # s = " "
  # output = s.join(stop_text)
  # print(output)


In [0]:
### Word embeddings
def word_embedding(data):
  model = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 5)
  print(model.similarity(data,'good'))

In [89]:
# open input file: 
ifile = open('/content/drive/My Drive/Musical_Instruments_5.json') 
all_data = list()
for i, line in enumerate(ifile): 
    # convert the json on this line to a dict
    data = json.loads(line)
    # extract what we want
    text = data['reviewText']
    prodid = data['asin']
    # add to the data collected so far
    all_data.append([prodid, text])
# create the DataFrame
df = pd.DataFrame(all_data, columns=['Product ID','Review'])
## Group by product ID
groupby_Id = df['Review'].groupby(df['Product ID'])
## z = No. of products
# z = len(list(groupby_Id))//100
z = 1
index = 28
for j in range(z):
  t = len(list(groupby_Id)[5][1])
  # print(t)
  for i in range(t):
    ### Text Preprocessing of reviews for each Product
    token_text = preprocess(list(groupby_Id)[5][1][index])
    # print('Pos Tag:',token_text)
    tag_list = []
    new_tokens = []
    for i in range(len(token_text)):
        tag_list.append(token_text[i][1])
    for i in range(len(tag_list)-1):
#### Adj noun combinations
       if ((tag_list[i] == 'JJ') and (tag_list[i+1] == 'NN' or tag_list[i] == 'NNP')):
            new_tokens.append(token_text[i][0])
            new_tokens.append(token_text[i+1][0])
#### noun/pronoun verb noun combinations
       if ((tag_list[i] == 'NN' or tag_list[i] == 'NNP') and (tag_list[i+1] == 'VB' or tag_list[i+1] == 'VBD' or  tag_list[i+1] == 'VBP')):
             new_tokens.append(token_text[i][0])
             new_tokens.append(token_text[i+1][0])
### noun/noun combinations
       if ((tag_list[i] == 'NN' or tag_list[i] == 'NNP') and (tag_list[i+1] == 'NN' or tag_list[i+1] == 'NNP')):
            new_tokens.append(token_text[i][0])
            new_tokens.append(token_text[i+1][0])
    if(np.isin('\n',new_tokens)):
      new_tokens.remove('\n')
    new_tokens = np.unique(new_tokens)
    print("New tokens: ",new_tokens)
    print('\n')   ##### Rule Based Aspects
    index += 1 
ifile.close()



['The Hosa XLR cables are affordable and very heavily made I have a large mixer and rack and cables everywhere I decided to purchase shorter cables and the Hosa cables 5ft measurement worked perfectly You really will not be disappointed with these ']
New tokens:  ['cable' 'decide' 'disappoint' 'hosa' 'large' 'measurement' 'mixer'
 'purchase' 'rack' 'short' 'work' 'xlr']


['I bought these to go from my board to the amp  We use them for a mobile church so they take a beating  They are still going strong ']
New tokens:  ['amp' 'board' 'church' 'mobile' 'strong' 'take' 'use']


["Sturdy cord and plugs inexpensive good value  I don't require professional-level equipment so this cord serves my purposes well  Satisfied with purchase "]
New tokens:  ['cord' 'equipment' 'good' 'plug' 'professional-level' 'purchase'
 'require' 'satisfy' 'serve' 'sturdy' 'value']


['Use it every week at gigs   Solid no problems with the solder joints   A good quality cable at a very good price ']
New tokens:  [